# **Business Data Science Project - Inside Airbnb (multi-city listings, reviews, calendars)**
### by Anastasia Bouev Dombre, Arthur Morvan, Aliénor Sabourdin, Elise Deyris

# Objective: Smarter pricing (predict how price affects bookings)


*   Idea: If a night is too expensive, fewer people book; too cheap, you leave money on the table.
* What we build: A model that predicts, for each date, how likely a night is to be booked at a given price.
* Output: Recommended price per date (e.g., “15 July: price ≈ 120 for ≈ 80% booking probability”).

For the first part concerning **Data Collection & Cleaning**
We will:
- Load the three CSV files(listings, calendar, reviews), we decided to focus on the city of Bordeaux, France.
- Inspect their structure
- Handle missing values (especially prices and reviews)
- Normalize price and availability fields
- Perform basic geospatial cleaning (coordinates and neighbourhood mapping)
- Export cleaned datasets for later feature engineering and modeling.

In [ ]:
#imports and paths
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

LISTINGS_PATH = "Airbnb-Listings-Data.csv"
CALENDAR_PATH = "Airbnb-Calendar-Data.csv"
REVIEWS_PATH = "Airbnb-Reviews.csv"


In [ ]:

listings_raw= pd.read_csv(LISTINGS_PATH)
calendar_raw= pd.read_csv(CALENDAR_PATH)
reviews_raw = pd.read_csv(REVIEWS_PATH)

print("Listings shape:", listings_raw.shape)
print("Calendar shape:", calendar_raw.shape)
print("Reviews shape:", reviews_raw.shape)


Listings shape: (12383, 79)
Calendar shape: (4519796, 7)
Reviews shape: (449626, 6)


In [ ]:
#displaying csv files info
print("\nListings head:")
display(listings_raw.head())
print("\nCalendar head:")
display(calendar_raw.head())
print("\nReviews head:")
display(reviews_raw.head())
print("\nListings info:")
listings_raw.info()
print("\nCalendar info:")
calendar_raw.info()
print("\nReviews info:")
reviews_raw.info()


Listings head:


,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,222887,https://www.airbnb.com/rooms/222887,20250615022817,2025-06-26,city scrape,"Spectacular view, full air-con, elevator",Imagine yourself relaxing on a 12m² private te...,✔ NEARBY MARKETS – Located near St Michel anti...,https://a0.muscache.com/pictures/8f2b210e-3c0e...,1156398,https://www.airbnb.com/users/show/1156398,Suzanne,2011-09-14,"Duras, France","Originally from New Zealand, I now divide my t...",within an hour,100%,94%,t,https://a0.muscache.com/im/pictures/user/User/...,https://a0.muscache.com/im/pictures/user/User/...,Hôtel de Ville - Quinconces,25,31,"['email', 'phone', 'work_email']",t,t,Neighborhood highlights,Bordeaux Sud,Bordeaux,44.836102,-0.566395,Entire vacation home,Entire home/apt,4,1.0,1 bath,2.0,3.0,"[""Shower gel"", ""Blender"", ""Waterfront"", ""Paid ...",$266.00,3,90,3.0,5.0,90.0,90.0,4.6,90.0,NaN,t,2,7,12,248,2025-06-26,127,31,5,77,28,186,49476.0,2012-01-04,2025-06-22,4.83,4.83,4.75,4.90,4.96,4.75,4.69,3306300031048,t,4,4,0,0,0.77
1,247452,https://www.airbnb.com/rooms/247452,20250615022817,2025-06-26,city scrape,"Cosy apartment ,barbecue, pool",NaN,NaN,https://a0.muscache.com/pictures/39503733/4b3c...,959918,https://www.airbnb.com/users/show/959918,Krista,2011-08-12,"Bordeaux, France","Hi, I'm Krista, 53 years old, Belgian and livi...",within a day,67%,69%,f,https://a0.muscache.com/im/users/959918/profil...,https://a0.muscache.com/im/users/959918/profil...,NaN,1,1,"['email', 'phone']",t,t,NaN,Saint-Mdard-en-Jalles,Saint-Mdard-en-Jalles,44.858900,-0.727350,Entire rental unit,Entire home/apt,4,1.0,1 bath,2.0,2.0,"[""Private pool"", ""Heating"", ""Wine glasses"", ""W...",$115.00,3,30,3.0,3.0,30.0,30.0,3.0,30.0,NaN,t,24,34,57,147,2025-06-26,79,9,0,147,12,54,6210.0,2012-04-23,2025-04-03,4.73,4.75,4.69,4.96,4.88,4.55,4.61,NaN,f,1,1,0,0,0.49
2,317273,https://www.airbnb.com/rooms/317273,20250615022817,2025-06-26,city scrape,"Luxury, spacious, patio, near public gardens",Bordeaux City Gardens is a luxurious one-bedro...,✔ PRIME LOCATION – Situated near the Bordeaux ...,https://a0.muscache.com/pictures/miso/Hosting-...,1156398,https://www.airbnb.com/users/show/1156398,Suzanne,2011-09-14,"Duras, France","Originally from New Zealand, I now divide my t...",within an hour,100%,94%,t,https://a0.muscache.com/im/pictures/user/User/...,https://a0.muscache.com/im/pictures/user/User/...,Hôtel de Ville - Quinconces,25,31,"['email', 'phone', 'work_email']",t,t,Neighborhood highlights,Chartrons - Grand Parc - Jardin Public,Bordeaux,44.847801,-0.581046,Entire rental unit,Entire home/apt,3,1.5,1.5 baths,1.0,2.0,"[""Paid parking garage 


Calendar head:


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,222887,2025-06-26,f,NaN,NaN,3,90
1,222887,2025-06-27,f,NaN,NaN,3,90
2,222887,2025-06-28,f,NaN,NaN,3,90
3,222887,2025-06-29,t,NaN,NaN,3,90
4,222887,2025-06-30,f,NaN,NaN,5,90



Reviews head:


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,222887,836020,2012-01-04,960075,Marjon,Suzanna's apartment is amazing and it was a tr...
1,222887,2717072,2012-10-25,2887504,Trevor,Great communication & directions from railway...
2,222887,2994458,2012-11-30,405763,William,Fabulous location over Bordeaux and manager ve...
3,222887,3561127,2013-02-17,169773,Daniel,Location is always a must for us and this plac...
4,222887,4230818,2013-04-21,4415172,Sue,This apartment is delightful. Thoughtfully an...



Listings info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12383 entries, 0 to 12382
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            12383 non-null  int64  
 1   listing_url                                   12383 non-null  object 
 2   scrape_id                                     12383 non-null  int64  
 3   last_scraped                                  12383 non-null  object 
 4   source                                        12383 non-null  object 
 5   name                                          12383 non-null  object 
 6   description                                   12095 non-null  object 
 7   neighborhood_overview                         6032 non-null   object 
 8   picture_url                                   12383 non-null  object 
 9   host_id                                      

Remarks:

* Price formatting: In the listings dataset, the price column is stored as strings like "$266.00" and contains missing values. We will convert it to numeric format and handle the missing entries.

* Boolean fields:host_is_superhost, host_has_profile_pic, and instant_bookable appear as "t,f". We will transform them in proper boolean values

* Missing values in texts: neighborhood_overview, host_about, and description have many missing values,  We will decide whether to impute them or ignore them for modeling

* Missing numeric values:bedrooms, beds, and bathrooms contain missing values

* Dates stored as strings: last_scraped, host_since, first_review... will be converted to proper datetime objects

* Geospatial cleaning: Since latitude and longitude are complete, we will filter listings to keep only those located within the Bordeaux area.

## Cleaning Listings Data

- Load the CSV file
- Clean the price column
- Filter to Bordeaux city boundaries (geospatial)
- Parse date columns to datetime
- Convert boolean columns
- Clean percentage columns
- Handle amenities and host_verifications (count items)
- Parse bathrooms column (extract numeric)
- Impute missing important numeric, categorical, and text values
- Format ID columns as strings
- Create date features (month, day of week)
- Remove duplicate listings

In [ ]:
import json
import re

listings = pd.read_csv("Airbnb-Listings-Data.csv")

# Clean the price column
if "price" in listings.columns:
    listings["price"] = listings["price"].replace(r'[\$,]', '', regex=True).astype(float)
    listings["price"] = listings["price"].fillna(listings["price"].median())

# Filter to Bordeaux city boundaries
# Remove listings outside Bordeaux bounding box
def bordeaux_bound(df, lat_col="latitude", lon_col="longitude"):
    mask = (df[lat_col].between(44.792, 44.890)) & (df[lon_col].between(-0.620, -0.520))
    return df[mask]
if {"latitude", "longitude"} <= set(listings.columns):
    listings =bordeaux_bound(listings)

# data columns in datetime
date_cols = ["last_scraped", "host_since", "first_review","last_review"]
for col in date_cols:
    if col in listings.columns:
        listings[col] = pd.to_datetime(listings[col],errors="coerce")

# convert boolean columns
def map_tf_to_bool(series):
    return series.map({'t': True, 'f': False, 'True': True, 'False': False}).astype('boolean')

bool_cols = [
    "host_is_superhost", "host_has_profile_pic", "host_identity_verified",
    "has_availability", "instant_bookable"
]
for col in bool_cols:
    if col in listings.columns:
        listings[col] = map_tf_to_bool(listings[col])

def clean_percentage(series):
    return series.str.replace('%', '', regex=False).astype(float)
for col in ['host_response_rate', 'host_acceptance_rate']:
    if col in listings.columns:
        listings[col] = clean_percentage(listings[col])

# handle amenities and host_verifications (count items), parse bathroom column,s
def count_items(x):
    if pd.isna(x) or x in ['[]', '']:
        return 0
    try:
        items = json.loads(x.replace("'", '"'))
        return len(items)
    except:
        return x.count(',') + 1

for col in ["amenities", "host_verifications"]:
    if col in listings.columns:
        listings[col] = listings[col].fillna('[]')
        listings[f"{col}_count"] =listings[col].apply(count_items)

def parse_bathrooms(text):
    if pd.isna(text): return None
    t = str(text).lower()
    if 'half' in t: return 0.5
    m = re.search(r'(\d+(\.\d+)?)', t)
    if m: return float(m.group(1))
    return None

if "bathrooms_text" in listings.columns:
    listings["bathrooms_clean"] = listings["bathrooms_text"].apply(parse_bathrooms)
elif "bathrooms" in listings.columns:
    listings["bathrooms_clean"] = pd.to_numeric(listings["bathrooms"], errors="coerce")
    listings["bathrooms_clean"] = listings["bathrooms_clean"].fillna(listings["bathrooms_clean"].median())

# handle missing values
num_cols = ["price", "accommodates", "bedrooms", "beds", "bathrooms_clean"]
for col in num_cols:
    if col in listings.columns:
        listings[col] = listings[col].fillna(listings[col].median())
cat_cols = ["room_type", "property_type", "neighbourhood_cleansed"]
for col in cat_cols:
    if col in listings.columns:
        listings[col] = listings[col].fillna(listings[col].mode()[0])
for col in ["description", "neighborhood_overview", "host_about"]:
    if col in listings.columns:
        listings[col] = listings[col].fillna("")

# format ID columns as strings
for col in ["id", "scrape_id", "host_id"]:
    if col in listings.columns:
        listings[col] = listings[col].astype(str)

# create date features (month, day of week)
if "last_scraped" in listings.columns:
    listings["month"] = listings["last_scraped"].dt.month
    listings["day_of_week"] = listings["last_scraped"].dt.dayofweek

# remove duplicates
if "id" in listings.columns:
    listings = listings.drop_duplicates(subset="id", keep="first")

# remove irrelevant columns that can be confusing for our upcoming analysis
columns_to_remove = [
    # URLs and images aren't needed
    'listing_url', 'host_url', 'picture_url', 
    'host_thumbnail_url', 'host_picture_url',
    
    # text fields that are irrelevant
    'description', 'neighborhood_overview', 'host_about', 'name',
    
    # redundant columns (we have cleaned/aggregated versions)
    'bathrooms_text',  # we have bathrooms_clean
    'bathrooms',  # we have bathrooms_clean
    'amenities',  # we have amenities_count
    'host_verifications',  # we have host_verifications_count
    
    # metadata that is not needed
    'scrape_id', 'source', 'last_scraped',  
    'calendar_last_scraped', 'calendar_updated',
    
    # redundant date features (we'll create from calendar dates instead)
    'month', 'day_of_week',  # these are from last_scraped, not useful
    
    # redundant availability (we have calendar data for date-level availability)
    'has_availability',  # we have date-level availability in calendar
    
    # other redundant columns
    'host_listings_count',  # we have calculated_host_listings_count
    'host_total_listings_count',  # redundant with calculated_host_listings_count
    'neighbourhood',  # we have neighbourhood_cleansed
    'host_neighbourhood',  # not needed
]

# Remove columns that exist
cols_to_drop = [col for col in columns_to_remove if col in listings.columns]
listings = listings.drop(columns=cols_to_drop)

print("CLEAN SHAPE:", listings.shape)
print(f"Removed {len(cols_to_drop)} irrelevant columns")
print(f"Remaining columns: {listings.shape[1]}")
listings.head()

CLEAN SHAPE: (9061, 59)
Removed 25 irrelevant columns
Remaining columns: 59


,id,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bedrooms,beds,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,availability_eoy,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,amenities_count,host_verifications_count,bathrooms_clean
0,222887,1156398,Suzanne,2011-09-14,"Duras, France",within an hour,100.0,94.0,True,True,True,Bordeaux Sud,Bordeaux,44.836102,-0.566395,Entire vacation home,Entire home/apt,4,2.0,3.0,266.0,3,90,3.0,5.0,90.0,90.0,4.6,90.0,2,7,12,248,127,31,5,77,28,186,49476.0,2012-01-04,2025-06-22,4.83,4.83,4.75,4.90,4.96,4.75,4.69,3306300031048,True,4,4,0,0,0.77,56,3,1.0
2,317273,1156398,Suzanne,2011-09-14,"Duras, France",within an hour,100.0,94.0,True,True,True,Chartrons - Grand Parc - Jardin Public,Bordeaux,44.847801,-0.581046,Entire rental unit,Entire home/apt,3,1.0,2.0,203.0,3,90,3.0,3.0,90.0,90.0,3.0,90.0,2,11,28,273,80,20,0,100,19,120,24360.0,2014-08-13,2025-04-29,4.90,4.98,4.83,4.96,4.96,4.96,4.78,33063001366CB,True,4,4,0,0,0.60,40,3,1.5
3,317658,1156398,Suzanne,2011-09-14,"Duras, France",within an hour,100.0,94.0,True,True,True,Centre ville (Bordeaux),Bordeaux,44.838799,-0.568870,Entire rental unit,Entire home/apt,6,2.0,4.0,222.0,3,90,2.0,5.0,90.0,90.0,4.5,90.0,11,17,27,269,161,17,0,93,21,102,22644.0,2012-03-15,2025-04-28,4.87,4.85,4.83,4.88,4.92,4.97,4.77,33063001225CF,True,4,4,0,0,1.00,48,3,1.0
4,333031,1697156,Antony,2012-02-04,"Bordeaux, France",within an hour,86.0,82.0,True,True,True,Centre ville (Bordeaux),Bordeaux,44.842560,-0.577940,Entire rental unit,Entire home/apt,2,0.0,1.0,103.0,1,1125,1.0,1.0,1125.0,1125.0,1.0,1125.0,24,54,82,353,556,53,4,178,47,255,26265.0,2012-04-02,2025-06-09,4.92,4.95,4.95,4.95,4.92,4.98,4.83,3306300055979,False,2,1,1,0,3.45,24,3,1.0
5,365993,1847986,Damien,2012-03-03,"Bordeaux, France",within a few hours,100.0,91.0,True,True,True,Bgles,Bgles,44.811490,-0.558250,Entire rental unit,Entire home/apt,6,2.0,3.0,77.0,5,30,5.0,5.0,30.0,30.0,5.0,30.0,7,20,34,221,72,9,2,45,6,90,6930.0,2012-05-27,2025-06-13,4.90,4.93,4.99,4.93,4.94,4.71,4.74,NaN,False,2,2,0,0,0.45,59,1,1.0


## Cleaning Calendar Data

The calendar data is critical for booking prediction as it contains:
- Date-level availability (available/booked labels)
- Price per date
- Minimum/maximum nights requirements

We will:
- Filter to Bordeaux listings (join with cleaned listings)
- Parse date column to datetime
- Convert available column to boolean
- Handle missing prices (use listing's base price or median)
- Create booking label (booked = not available)
- Remove invalid dates


In [22]:
# --- Load calendar data ---
calendar = pd.read_csv(CALENDAR_PATH)

print("Calendar raw shape:", calendar.shape)
print("\nCalendar columns:", calendar.columns.tolist())
print("\nCalendar dtypes:")
print(calendar.dtypes)
print("\nCalendar head:")
display(calendar.head())
print("\nMissing values:")
print(calendar.isnull().sum())
print("\nAvailable values unique:", calendar['available'].unique() if 'available' in calendar.columns else "N/A")


Calendar raw shape: (4519796, 7)

Calendar columns: ['listing_id', 'date', 'available', 'price', 'adjusted_price', 'minimum_nights', 'maximum_nights']

Calendar dtypes:
listing_id          int64
date               object
available          object
price             float64
adjusted_price    float64
minimum_nights      int64
maximum_nights      int64
dtype: object

Calendar head:


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,222887,2025-06-26,f,NaN,NaN,3,90
1,222887,2025-06-27,f,NaN,NaN,3,90
2,222887,2025-06-28,f,NaN,NaN,3,90
3,222887,2025-06-29,t,NaN,NaN,3,90
4,222887,2025-06-30,f,NaN,NaN,5,90



Missing values:
listing_id              0
date                    0
available               0
price             4519796
adjusted_price    4519796
minimum_nights          0
maximum_nights          0
dtype: int64

Available values unique: ['f' 't']


In [29]:
# clean calandar data

# filter to Bordeaux listings only
calendar['listing_id'] = calendar['listing_id'].astype(str)
bordeaux_listing_ids = set(listings['id'].astype(str))
calendar = calendar[calendar['listing_id'].isin(bordeaux_listing_ids)].copy()

# parse date and convert available to boolean
calendar['date'] = pd.to_datetime(calendar['date'], errors='coerce')
calendar = calendar[calendar['date'].notna()].copy()

calendar['available'] = calendar['available'].map({'t': True, 'f': False, 'True': True, 'False': False, True: True, False: False})
calendar['available'] = calendar['available'].astype('boolean')
calendar['is_booked'] = ~calendar['available']  # booked = not available

# handle prices - chain: adjusted_price -> price -> listing price -> median
listings_price_map = listings.set_index('id')['price'].to_dict()
calendar['listing_base_price'] = calendar['listing_id'].map(listings_price_map)

# create price_clean using fillna chain
if 'adjusted_price' in calendar.columns and 'price' in calendar.columns:
    calendar['price_clean'] = calendar['adjusted_price'].fillna(calendar['price']).fillna(calendar['listing_base_price'])
elif 'adjusted_price' in calendar.columns:
    calendar['price_clean'] = calendar['adjusted_price'].fillna(calendar['listing_base_price'])
elif 'price' in calendar.columns:
    calendar['price_clean'] = calendar['price'].fillna(calendar['listing_base_price'])
else:
    calendar['price_clean'] = calendar['listing_base_price']

# fill remaining missing with median
if calendar['price_clean'].notna().sum() > 0:
    calendar['price_clean'] = calendar['price_clean'].fillna(calendar['price_clean'].median())

# remove rows with missing prices
calendar = calendar[calendar['price_clean'].notna()].copy()

#clean up
calendar = calendar.dropna(subset=['listing_id', 'date', 'available'])
calendar = calendar.sort_values(['listing_id', 'date']).reset_index(drop=True)

#remove redundant columns
cols_to_remove =['price', 'adjusted_price', 'listing_base_price']
cols_to_remove = [col for col in cols_to_remove if col in calendar.columns]
calendar = calendar.drop(columns=cols_to_remove)

#display results
print(f"Final calendar shape: {calendar.shape}")
print(f"Date range: {calendar['date'].min()} to {calendar['date'].max()}")
print(f"Booking rate: {calendar['is_booked'].mean():.2%}")
print(f"Price range: {calendar['price_clean'].min():.2f} - {calendar['price_clean'].max():.2f}")
calendar.head(30)


Final calendar shape: (3307266, 7)
Date range: 2025-06-25 00:00:00 to 2026-06-26 00:00:00
Booking rate: 63.26%
Price range: 12.00 - 10000.00


,listing_id,date,available,minimum_nights,maximum_nights,is_booked,price_clean
0,1000537871351948755,2025-06-26,False,1,365,True,76.0
1,1000537871351948755,2025-06-27,False,1,365,True,76.0
2,1000537871351948755,2025-06-28,False,1,365,True,76.0
3,1000537871351948755,2025-06-29,True,1,365,False,76.0
4,1000537871351948755,2025-06-30,True,1,365,False,76.0
5,1000537871351948755,2025-07-01,False,1,365,True,76.0
6,1000537871351948755,2025-07-02,True,1,365,False,76.0
7,1000537871351948755,2025-07-03,True,1,365,False,76.0
8,1000537871351948755,2025-07-04,True,1,365,False,76.0
9,1000537871351948755,2025-07-05,True,1,365,False,76.0


# Clean Reviews Data

We will:
- Filter to Bordeaux listings 
- Parse date column to datetime
- Handle missing prices 
- Sort by listing_id and date
- Remove irrelevant columns


In [30]:
# --- Load reviews data ---
reviews = pd.read_csv(REVIEWS_PATH)

print("Reviews raw shape:", reviews.shape)
print("\nReviews columns:", reviews.columns.tolist())
print("\nReviews dtypes:")
print(reviews.dtypes)
print("\nMissing values:")
print(reviews.isnull().sum())
print("\nReviews head:")
display(reviews.head())


Reviews raw shape: (449626, 6)

Reviews columns: ['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments']

Reviews dtypes:
listing_id        int64
id                int64
date             object
reviewer_id       int64
reviewer_name    object
comments         object
dtype: object

Missing values:
listing_id       0
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         9
dtype: int64

Reviews head:


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,222887,836020,2012-01-04,960075,Marjon,Suzanna's apartment is amazing and it was a tr...
1,222887,2717072,2012-10-25,2887504,Trevor,Great communication & directions from railway...
2,222887,2994458,2012-11-30,405763,William,Fabulous location over Bordeaux and manager ve...
3,222887,3561127,2013-02-17,169773,Daniel,Location is always a must for us and this plac...
4,222887,4230818,2013-04-21,4415172,Sue,This apartment is delightful. Thoughtfully an...


In [31]:
# --- Clean Reviews Data ---

# filter to Bordeaux listings only
reviews['listing_id'] = reviews['listing_id'].astype(str)
reviews = reviews[reviews['listing_id'].isin(bordeaux_listing_ids)].copy()

print(f"Reviews after filtering to Bordeaux: {reviews.shape}")

# parse date column to datetime
if 'date' in reviews.columns:
    reviews['date'] = pd.to_datetime(reviews['date'], errors='coerce')
    # Remove rows with invalid dates
    reviews = reviews[reviews['date'].notna()].copy()

# handle missing comments (fill with empty string for now, can be used for sentiment analysis later)
if 'comments' in reviews.columns:
    reviews['comments'] = reviews['comments'].fillna('')


# remove any remaining invalid rows
reviews = reviews.dropna(subset=['listing_id', 'date'])

# sort by listing_id and date
reviews = reviews.sort_values(['listing_id', 'date']).reset_index(drop=True)

# remove irrelevant columns
columns_to_remove_reviews = [
    'reviewer_name',  # not needed for modeling
    'reviewer_id',  # not needed unless analyzing reviewer behavior
    'id',  # review ID not needed (we have listing_id and date)
]

cols_to_drop_rev = [col for col in columns_to_remove_reviews if col in reviews.columns]
reviews = reviews.drop(columns=cols_to_drop_rev)

print(f"\nFinal reviews shape: {reviews.shape}")
print(f"Removed {len(cols_to_drop_rev)} irrelevant columns")
print(f"\nDate range:{reviews['date'].min()} to {reviews['date'].max()}")
print(f"\nReviews with comments: {(reviews['comments'].str.strip() != '').sum()}")
print(f"\nSample of cleaned reviews:")
reviews.head(10)


Reviews after filtering to Bordeaux: (351940, 6)

Final reviews shape: (351940, 3)
Removed 3 irrelevant columns

Date range:2012-01-04 00:00:00 to 2025-06-26 00:00:00

Reviews with comments: 351933

Sample of cleaned reviews:


,listing_id,date,comments
0,1000537871351948755,2023-11-19,Tout s’est très bien passé ! Appartement très ...
1,1000537871351948755,2023-11-26,Ha sido un placer alojarnos con Camille.<br/>H...
2,1000537871351948755,2023-12-10,Je remercie Camille pour son accueil dans l'ap...
3,1000537871351948755,2023-12-15,J'ai passé un court mais excellent sejour chez...
4,1000537871351948755,2024-02-04,Hôte disponible <br/>Logement très accueillant...
5,1000537871351948755,2024-02-18,"Hôte très accueillante , très disponible , log..."
6,1000537871351948755,2024-03-19,"Airbnb, parfait, j’y retournerai sans hésiter !"
7,1000537871351948755,2024-04-02,Merci à Camille pour ce séjour fabuleux ! Le l...
8,1000537871351948755,2024-04-28,"Nous avons passé deux nuits à l'échoppe, l'env..."
9,1000537871351948755,2024-05-06,"Super séjour à l’échoppe, très spacieux et bie..."


In [32]:
# Summuary of Cleaned Datasets


print(f"LISTINGS:")
print(f"Shape: {listings.shape}")
print(f"   Key columns: id, host_id, price, room_type, property_type,")
print(f"                accommodates, bedrooms, beds, bathrooms_clean,")
print(f"                amenities_count, neighbourhood_cleansed, latitude, longitude,")
print(f"                review_scores_*, host_metrics,...")

print("\n")
print(f"CALENDAR:")
print(f"   Shape: {calendar.shape}")
print(f"   Key columns: listing_id, date, available, is_booked, price_clean,")
print(f"                minimum_nights, maximum_nights")

print("\n")
print(f"REVIEWS:")
print(f"   Shape: {reviews.shape}")
print(f"   Key columns: listing_id, date, comments")



LISTINGS:
Shape: (9061, 59)
   Key columns: id, host_id, price, room_type, property_type,
                accommodates, bedrooms, beds, bathrooms_clean,
                amenities_count, neighbourhood_cleansed, latitude, longitude,
                review_scores_*, host_metrics,...


CALENDAR:
   Shape: (3307266, 7)
   Key columns: listing_id, date, available, is_booked, price_clean,
                minimum_nights, maximum_nights


REVIEWS:
   Shape: (351940, 3)
   Key columns: listing_id, date, comments


## Summary of Data Cleaning

**Listings Data**: Cleaned and filtered to Bordeaux 
- Price normalized, dates parsed, booleans converted, missing values handled, duplicates removed

**Calendar Data**: Cleaned and filtered to Bordeaux listings, 
- Date level availability and prices ready for booking prediction, booking labels created (is_booked = not available), missing prices handled

**Reviews Data**: Cleaned and filtered to Bordeaux listings
- Ready for review-based feature engineering

# Next Step : Feature Engineering and Modeling !
